# **Exporting distances of edges from given road segments**

In [1]:
MAX_DISTANCE = 150

In [2]:
import os, sys
import numpy as np
import json

SUMO_HOME = os.environ["SUMO_HOME"] #locating the simulator
sys.path.append(SUMO_HOME+"/tools")
import traci
import sumolib

In [3]:
MOST_PATH = "../../MoSTScenario/scenario/most.sumocfg"
PARKING_AREA_DEFINITIONS = "../../MoSTScenario/scenario/in/add/most.parking.add.xml"

In [4]:
import pandas as pd

parking_df = pd.read_xml(PARKING_AREA_DEFINITIONS)
parking_df = parking_df[parking_df["roadsideCapacity"]>0]

In [5]:
sumoBinary = "sumo"
sumoCmd = [sumoBinary, "-c", MOST_PATH,
           "--random", "--random-depart-offset", "900", "--verbose", "false",
           "--net-file", "../../MoSTScenario/scenario/in/most.net.xml"]
sumoCmd += ["--additional-files", "../../MoSTScenario/scenario/in/add/most.poly.xml,../../MoSTScenario/scenario/in/add/most.parking.allvisible.add.xml"]

traci.start(sumoCmd)

 Retrying in 1 seconds


sumo: /home/madbiker/miniconda3/envs/tf_env/lib/libtiff.so.5: no version information available (required by /lib/x86_64-linux-gnu/libproj.so.22)
sumo: /home/madbiker/miniconda3/envs/tf_env/lib/libtiff.so.5: no version information available (required by /lib/x86_64-linux-gnu/libFOX-1.6.so.0)


(20, 'SUMO 1.15.0')

In [6]:
parking_ids = parking_df["id"].values.tolist()
parking_lanes = parking_df["lane"].values.tolist()
parking_edges = []
for i in range(len(parking_lanes)):
    parking_edges.append(traci.lane.getEdgeID(parking_lanes[i]))

In [7]:
edges = traci.edge.getIDList()

distances = {}

for i in range(len(parking_ids)):
    for j in range(len(edges)):
        distance_ = traci.simulation.getDistanceRoad(parking_edges[i],
            traci.lane.getLength(parking_lanes[i])/2,
            edges[j],
            traci.lane.getLength(edges[j]+"_0")/2)
        if distance_ <= MAX_DISTANCE:
            if not(parking_ids[i] in distances):
                distances[parking_ids[i]] = []
            distances[parking_ids[i]].append(edges[j])

In [8]:
with open("../02_data/proximity.json", "w") as f:
    json.dump(distances, f)

In [9]:
traci.close()

Step #14400.00 (0ms ?*RT. ?UPS, TraCI: 262812ms, vehicles TOT 0 ACT 0 BUF 0)              


# **Utilizing the distances to create measurement files**

In [10]:
DAY = 0
MEASUREMENT_PATH = "/media/madbiker/little_j/day_%d/"%DAY

In [11]:
import os, sys
import numpy as np
import json
import pandas as pd

In [12]:
MOST_PATH = "../../MoSTScenario/scenario/most.sumocfg"
PARKING_AREA_DEFINITIONS = "../../MoSTScenario/scenario/in/add/most.parking.add.xml"

parking_df = pd.read_xml(PARKING_AREA_DEFINITIONS).set_index("id")
parking_df = parking_df[parking_df["roadsideCapacity"]>0]

In [13]:
parking_df

,lane,startPos,endPos,roadsideCapacity,friendlyPos,edges,interval
id,,,,,,,
1059,-153419_1,10.000000,20.000000,225.0,true,None,NaN
1060,-152335_1,18.191499,28.191499,75.0,true,None,NaN
1061,153427#0_1,10.000000,20.000000,350.0,true,None,NaN
1062,-153427#5_1,10.583918,20.583918,350.0,true,None,NaN
1063,153547_1,37.879874,47.879874,50.0,true,None,NaN
...,...,...,...,...,...,...,...
1182,152858_1,10.000000,20.000000,75.0,true,None,NaN
1183,153323#4_1,197.261809,207.261809,250.0,true,None,NaN
1184,-153323#1_1,31.065872,41.065872,75.0,true,None,NaN


In [15]:
with open("../02_data/proximity.json", "r") as f:
    distances = json.load(f)
    
proximity = {}
for x in distances:
    for y in distances[x]:
        if not(y in proximity):
            proximity[y] = []
        proximity[y].append(x)

In [17]:
with open(MEASUREMENT_PATH+"edge_maps.json", "r") as f:
    maps = json.load(f)
    edge_to_idx = maps["edge_to_idx_map"]
    idx_to_edge = maps["idx_to_edge_map"]

In [18]:
movements = pd.read_csv(MEASUREMENT_PATH+"_vehicle_positions.csv")

In [19]:
occupancies = pd.read_csv(MEASUREMENT_PATH+"occupancies.csv")

In [22]:
occupancies

,ids,timestamp,counts
0,1059,14460,0
1,1060,14460,0
2,1061,14460,0
3,1062,14460,0
4,1063,14460,0
...,...,...,...
152395,1182,50400,0
152396,1183,50400,0
152397,1184,50400,0
152398,1185,50400,0


In [37]:
test = occupancies[occupancies["ids"] == 1120]
test[test["timestamp"] == 18060].counts.iloc[0]

0

In [ ]:
with open(MEASUREMENT_PATH+"vehicle_measurement.csv", "w") as f:
    f.write("veh_id,day,timestamp,parking_id,occupancy\n")
    for i,r in movements.iterrows():
        edge_id = idx_to_edge[str(r["position"])]
        if edge_id in proximity:
            for p_lot in proximity[edge_id]:
                time = int((r.timestamp // 60)*60)
                count = occupancies[occupancies["timestamp"] == time]
                count = count[count["ids"] == int(p_lot)].counts
                occupancy = count.iloc[0] / parking_df.loc[int(p_lot)]["roadsideCapacity"]
                f.write("%d,%d,%d,%d,%f\n"%(
                    r.ids, DAY, r.timestamp, int(p_lot), occupancy
                ))